# Create cylinders
This note book uses MCMRSimulator v0.9.0 and custom function `repel_fixed_radius()` in `repel_cylinders.jl` to generate parallel cylinder substrates for our simulation. The custom function is needed because MCMRSimulator's built-in function `random_positions_radii()` generated cylinder packing configurations that were highly inhomogeneous, which resulted in clusters of cylinders and large gaps between clusters. This affected the consistency and accuracy of our simulated signal. `repel_fixed_radius()` takes the output of `random_positions_radii()` and add repulsions between cylinders to create a more homogeneous packing configuration. It returns the positions and radius of all cylinders in a block. The `Cylinders()` constructor then creates the cylinder object in the simulation with the given position and radius and infinitely repeats the block to fill in all space. In later versions of MCMRSimulator, such as the v0.11.0 we used for the example simulation, the custom function has been integrated into the `random_positions_radii()`, therefore using `random_positions_radii()` alone is sufficient.

In [2]:
# Load the pkg module to activate the correct environment
using Pkg
Pkg.activate(".")
Pkg.instantiate()
# import the relevant packages
using MCMRSimulator
using CairoMakie
using Statistics
using DelimitedFiles
using FileIO
using Printf
using JLD2
include("../repel_cylinders.jl")

  Activating project at `~/Library/CloudStorage/OneDrive-Nexus365/SWI&DTI/Papers/2024 MRM/gitrepo/MT-and-permeability-effect-on-two-compartment-dMRI-WM-model/fixed_diameter`


repel_distributed_radius

In [6]:
T2s = 10:10:150       # Define effective T2 (MT strength) range (ms)
T2_bound=1e-3         # Bound pool T2 to model surface relaxivity (ms)
rho=0.65              # Cylinder volume density
r0 = 0.3              # Lowest cylinder radius 
rep = r0*100          # (half of) the distance over which the substrate repeats itself spatially
t_dwell = 30          # Dwell time of isochromats in the bound pool, controls the strength of surface relaxivity

# Create a substrate with parallel cylinders of fixed diameters
# Determine the positions of cylinder centres for the smallest cylinder radius case
res = MCMRSimulator.random_positions_radii([rep*1.001, rep*1.001], rho*1.001^2, 2, mean=r0*1.001, variance=0)
repeled = repel_fixed_radius(r0*1.001, res[1], rep*1.001, maxiter=1000, repulsion_strength=3e-3) 

# Scale all the larger cylinders from the same configuration for consistency
for r in 0.3:0.1:5.5          # Loop through the desired radius range
    rep = r*100               # Scale the size of the block of substrates that gets repeated
    svratio = 2/r             # Calculate the surface to volume ratio that is used to calculate the surface density later
    for T2 = T2s              # Loop through the desired MT range
        surf_dens = 1/svratio*t_dwell/T2        # Surface density of isochromats that achieves the given T2, it's the ratio of isochromat density on the substrate (cylinder) and isochromat density in the volume of interest
        rng_geom = Cylinders(position=repeled.*(r/r0), radius=r, dwell_time=t_dwell, density=surf_dens, repeats=[rep, rep], R2_surface=1/T2_bound)           # Create the cylinder substrate object
        MCMRSimulator.write_geometry("./MT_cyl/cylinders_MT_"* string(T2) *"_sus_0_perm_0.000_rmean_"*@sprintf("%.2f",r)*"_density_0.65.json", rng_geom)     # Save the substrate object
    end
    for perm in 0:0.001:0.02  # Loop through the desired permeability range
        rng_geom = Cylinders(position=repeled.*(r/r0), radius=r, permeability=perm, repeats=[rep, rep])                                                             # Create the cylinder substrate object
        MCMRSimulator.write_geometry("./perm_cyl/cylinders_MT_0_sus_0_perm_"*@sprintf("%.3f",perm)*"_rmean_"*@sprintf("%.2f",r)*"_density_0.65.json", rng_geom)     # Save the substrate object
    end
end


There are 2074 Cylinder based on values for Field(position)
There are 2074 Cylinder based on values for Field(position)
There are 2074 Cylinder based on values for Field(position)
There are 2074 Cylinder based on values for Field(position)
There are 2074 Cylinder based on values for Field(position)
There are 2074 Cylinder based on values for Field(position)
There are 2074 Cylinder based on values for Field(position)
There are 2074 Cylinder based on values for Field(position)
There are 2074 Cylinder based on values for Field(position)
There are 2074 Cylinder based on values for Field(position)
There are 2074 Cylinder based on values for Field(position)
There are 2074 Cylinder based on values for Field(position)
There are 2074 Cylinder based on values for Field(position)
There are 2074 Cylinder based on values for Field(position)
There are 2074 Cylinder based on values for Field(position)
There are 2074 Cylinder based on values for Field(position)
There are 2074 Cylinder based on values 